# Relaxed Search(제대로) 가미한 코드

아는 선에서 일단 건드려봤고 1차적인 결과를 올림<br>
알고리즘 돌리는 코드들은 동환이형이 쓴 그대로 썼고,<br>
알고리즘에 따른 셀 분리, relaxed search, best model 등을 구하기만 함. <br>
한번 쭉 봐주고 갖고 놀아보고 이것저것 만져보면 뭐가 뭔지 대충 느낌 올거임.<br>
아님 말구<br>
<strong>plot 은 아직 하나도 안 넣음(ex. 동환이형의 시간축 x 그래프). data visualization 에 필수적인 것이니 꼭 넣어야함</strong>
<br><br>

### 5월 23일 업데이트

<strong> 이 코드 예상 실행 시간... 적으면 2분... 많으면 5분?(데스크탑 기준...)</strong><br>
데이터 양에 따라 차이가 남.


In [1]:
# 최적의 패러미터 가진 모델을 몇번 돌릴지 값.
test_times=10
# random 인 max_state 를 설정하기 위한 변수.
full_count=0 # 건들지 말것.

In [2]:
#데이터 파일 업로드
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from numpy import mean
from numpy import std
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import scipy.stats
import random
random.seed(42)
import warnings
np.set_printoptions(precision=4,suppress=True)
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',500)
plt.rcParams["font.family"]='Malgun Gothic'

아래 셀에서 데이터 수정. <strong> 앞에 Data/ 있어야 제대로 됨</strong><br>
원하는 데이터 이름만 집어넣으면 뒤에는 알아서 될거임

In [3]:
dataname = "ilsan_3years_small.csv"

In [4]:
#데이터 확인 및 필요 없는 열 삭제
pd.set_option('display.max_columns', 500)
df=pd.read_csv(dataname)
df.drop(['시군구'],axis=1,inplace=True)
df.drop(['단지명'],axis=1,inplace=True)
df.drop(['건축년도'],axis=1,inplace=True)

Y = df['거래금액'].values
Y = Y.astype('float')
X = df.drop(labels=['거래금액'],axis=1)
X = X.astype('float')

In [5]:
test_size = [0.1,0.15,0.2,0.25,0.3]
ts_len = len(test_size)

### 1. Random Forest

n_estimators = number of trees in the foreset<br>
max_features = max number of features considered for splitting a node<br>
max_depth = max number of levels in each decision tree<br>
min_samples_split = min number of data points placed in a node before the node is split<br>
min_samples_leaf = min number of data points allowed in a leaf node<br>
bootstrap = method for sampling data points (with or without replacement)

In [11]:
Tree_nums = [5,10,50,100,250,500,750]
Var_nums = [1,3,5,7]

# Random Forest Relaxed Search
x = len(Tree_nums)
y = len(Var_nums)
count=0

RF_RMSE_train_list =[]
RF_RMSE_test_list =[]
RF_MAPE_train_list =[]
RF_MAPE_test_list =[]
RF_par_diff_list=[]
RF_par_tsize_list=[]
RF_par_trees_list=[]
RF_par_var_list=[]
RF_min_vars_tsize_list=[]
RF_min_vars_trees_list=[]
RF_min_vars_var_list=[]   

print("----progress...----")

for ts in test_size:
    for a in Tree_nums:
        for b in Var_nums:
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts, random_state=full_count)
        
            RF_model = RandomForestRegressor(n_estimators=a, max_depth=b,random_state=full_count)
            RF_model.fit(X_train, Y_train)
            
            RF_prediction_test = RF_model.predict(X_test)
            RF_prediction_test = RF_prediction_test.astype('int')
            
            RF_train_test = RF_model.predict(X_train)
            RF_train_test = RF_train_test.astype('int')
            
            RF_RMSE_train =np.sqrt(mean_squared_error(Y_train, RF_train_test)).astype('int')
            RF_MAPE_train = round(np.mean(np.abs((Y_train - RF_train_test) / Y_train)) * 100 ,2)
            

            RF_RMSE_test = np.sqrt(mean_squared_error(Y_test, RF_prediction_test)).astype('int')
            RF_MAPE_test = round(np.mean(np.abs((Y_test - RF_prediction_test) / Y_test)) * 100 ,2)
            

            RF_RMSE_train_list.append(RF_RMSE_train)
            RF_RMSE_test_list.append(RF_RMSE_test)
            RF_MAPE_train_list.append(RF_MAPE_train)
            RF_MAPE_test_list.append(RF_MAPE_test)
            
            RF_RMSE_diff = np.abs((RF_RMSE_test-RF_RMSE_train)/RF_RMSE_test *100)
            
            RF_par_diff_list.append(RF_RMSE_diff)
            RF_par_tsize_list.append(ts)
            RF_par_trees_list.append(a)
            RF_par_var_list.append(b)
                
            full_count+=1
            count+=1
            progress = round(count/(x*y*ts_len)*100,2)
            
            print(" {:.2f}% , case:{} , test_size: {} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
            "---  train RMSE : {} , test RMSE : {}\n"
            "---  train MAPE : {} , test MAPE : {}"
                .format(progress,count,ts,a,b,RF_RMSE_diff,RF_RMSE_train,RF_RMSE_test,RF_MAPE_train,RF_MAPE_test))
            

for i in range(5):
    val = min(RF_par_diff_list)
    index = RF_par_diff_list.index(val)
    RF_min_vars_tsize_list.append(RF_par_tsize_list.pop(index))
    RF_min_vars_trees_list.append(RF_par_trees_list.pop(index))
    RF_min_vars_var_list.append(RF_par_var_list.pop(index))


----progress...----
 0.71% , case:1 , test_size: 0.1 , Tree_num: 5 , Var_num: 1 , difference(%) : 2.94
 ---  train RMSE : 2732 , test RMSE : 2654
---  train MAPE : 17.17 , test MAPE : 18.1
 1.43% , case:2 , test_size: 0.1 , Tree_num: 5 , Var_num: 3 , difference(%) : 3.39
 ---  train RMSE : 1830 , test RMSE : 1770
---  train MAPE : 11.27 , test MAPE : 12.62
 2.14% , case:3 , test_size: 0.1 , Tree_num: 5 , Var_num: 5 , difference(%) : 11.48
 ---  train RMSE : 1365 , test RMSE : 1542
---  train MAPE : 8.7 , test MAPE : 7.89
 2.86% , case:4 , test_size: 0.1 , Tree_num: 5 , Var_num: 7 , difference(%) : 10.70
 ---  train RMSE : 985 , test RMSE : 1103
---  train MAPE : 5.67 , test MAPE : 6.84
 3.57% , case:5 , test_size: 0.1 , Tree_num: 10 , Var_num: 1 , difference(%) : 1.84
 ---  train RMSE : 2767 , test RMSE : 2819
---  train MAPE : 17.68 , test MAPE : 16.49
 4.29% , case:6 , test_size: 0.1 , Tree_num: 10 , Var_num: 3 , difference(%) : 15.49
 ---  train RMSE : 1827 , test RMSE : 1582
---  t

In [7]:
print("PARAMETERS FOR BEST RF...")
for i in range(5):
    print("test_size : {} , Tree_num : {}, Var_num : {}".format(RF_min_vars_tsize_list[i],RF_min_vars_trees_list[i],RF_min_vars_var_list[i]))

print("...testing...")

progress=0
count=0
RF_Best_RMSE_train_list=[]
RF_Best_RMSE_test_list=[]
RF_Best_MAPE_train_list=[]
RF_Best_MAPE_test_list=[]
RF_Best_RMSE_diff_list=[]

RF_Best_total_data=[]

for i in range(5):
    temp_RF_Best_RMSE_train_list=[]
    temp_RF_Best_RMSE_test_list=[]
    temp_RF_Best_MAPE_train_list=[]
    temp_RF_Best_MAPE_test_list=[]
    temp_RF_Best_RMSE_diff_list=[]
    for k in range(test_times):
        ts = RF_min_vars_tsize_list[i]
        a = RF_min_vars_trees_list[i]
        b = RF_min_vars_var_list[i]

        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts,random_state=full_count)

        RF_Best_model = RandomForestRegressor(n_estimators=a, max_depth=b,random_state=full_count )
        RF_Best_model.fit(X_train, Y_train)

        RF_Best_prediction_test = RF_Best_model.predict(X_test)
        RF_Best_prediction_test = RF_Best_prediction_test.astype('int')

        RF_Best_train_test = RF_Best_model.predict(X_train)
        RF_Best_train_test = RF_Best_train_test.astype('int')

        RF_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, RF_Best_train_test)).astype('int')
        RF_Best_MAPE_train = round(np.mean(np.abs((Y_train - RF_Best_train_test) / Y_train)) * 100 ,2)

        RF_Best_RMSE_test = np.sqrt(mean_squared_error(Y_test, RF_Best_prediction_test)).astype('int')
        RF_Best_MAPE_test = round(np.mean(np.abs((Y_test - RF_Best_prediction_test) / Y_test)) * 100 ,2)

        RF_Best_RMSE_diff = np.abs((RF_Best_RMSE_test-RF_Best_RMSE_train)/RF_Best_RMSE_test *100)
        
        temp_RF_Best_RMSE_train_list.append(RF_Best_RMSE_train)
        temp_RF_Best_RMSE_test_list.append(RF_Best_RMSE_test)
        temp_RF_Best_MAPE_train_list.append(RF_Best_MAPE_train)
        temp_RF_Best_MAPE_test_list.append(RF_Best_MAPE_test)
        temp_RF_Best_RMSE_diff_list.append(RF_Best_RMSE_diff)
        
        full_count+=1
        count+=1
        progress = round(count/(10*5)*100,2)
#         print(" {:.2f}% , case:{} , test_size: {} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
#                 "---  train RMSE : {} , test RMSE : {}\n"
#                 "---  train MAPE : {} , test MAPE : {}"
#                       .format(progress,count,ts,a,b,RF_Best_RMSE_diff,RF_Best_RMSE_train,RF_Best_RMSE_test,RF_Best_MAPE_train,RF_Best_MAPE_test))
    
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_train_list))
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_test_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_train_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_test_list))
    RF_Best_total_data.append(mean(temp_RF_Best_MAPE_train_list))
    RF_Best_total_data.append(mean(temp_RF_Best_MAPE_test_list))
    RF_Best_total_data.append(std(temp_RF_Best_MAPE_train_list))
    RF_Best_total_data.append(std(temp_RF_Best_MAPE_test_list))
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_diff_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_diff_list))


PARAMETERS FOR BEST RF...
test_size : 0.1 , Tree_num : 50, Var_num : 3
test_size : 0.1 , Tree_num : 50, Var_num : 5
test_size : 0.1 , Tree_num : 50, Var_num : 7
test_size : 0.1 , Tree_num : 100, Var_num : 1
test_size : 0.1 , Tree_num : 100, Var_num : 3
...testing...


In [8]:
rf1=[]
rf2=[]
rf3=[]
rf4=[]
rf5=[]

for i in range(50):
    k=i//10
    if k==0:
        rf1.append(RF_Best_total_data[i])
    if k==1:
        rf2.append(RF_Best_total_data[i])
    if k==2:
        rf3.append(RF_Best_total_data[i])
    if k==3:
        rf4.append(RF_Best_total_data[i])
    if k==4:
        rf5.append(RF_Best_total_data[i])
    
RF_s1 = pd.Series(rf1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

RF_s2 = pd.Series(rf2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
RF_s3 = pd.Series(rf3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
RF_s4 = pd.Series(rf4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
RF_s5 = pd.Series(rf5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')

RF_best_total_dataframe = pd.concat([RF_s1,RF_s2,RF_s3,RF_s4,RF_s5],axis=1)
RF_best_total_dataframe

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,1809.100000,1297.500000,925.300000,2727.100000,1807.900000
RMSE test Mean,1833.800000,1552.300000,1146.200000,2632.400000,1863.200000
RMSE train Stdev,14.432256,28.040150,15.874823,20.525350,16.837161
RMSE test Stdev,166.630609,368.764708,92.016086,148.164233,252.648293
MAPE train Mean,11.424000,7.948000,5.448000,17.256000,11.436000
MAPE test Mean,11.611000,8.473000,6.714000,17.059000,11.112000
MAPE train Stdev,0.139442,0.127969,0.086810,0.115689,0.132303
MAPE test Stdev,0.619717,0.354177,0.362938,0.906040,0.710631
RMSE diff(%) Mean,8.508175,13.729660,18.656721,5.840756,8.702250
RMSE diff(%) Stdev,4.979298,15.924460,7.705873,5.111524,7.799640


In [ ]:
RMSE train Mean	1793.600000	1305.900000	915.700000	2721.300000	1840.100000
RMSE test Mean	1932.700000	1413.700000	1317.500000	2838.300000	1975.500000
RMSE train Stdev	20.819222	14.815195	21.014519	81.928078	32.163489
RMSE test Stdev	255.896483	185.039482	458.457686	351.154681	115.239099
MAPE train Mean	11.364000	7.992000	5.460000	17.340000	11.592000
MAPE test Mean	11.233000	8.512000	6.401000	17.851000	11.954000
MAPE train Stdev	0.110109	0.086232	0.075498	0.244459	0.232800
MAPE test Stdev	1.072996	0.778021	0.372195	0.926288	0.570968
RMSE diff(%) Mean	11.464243	12.284664	24.161033	10.371810	6.527199
RMSE diff(%) Stdev	8.464881	7.361594	19.611445	8.688818	5.789716

### 2. Support Vector Machine parameters

In [ ]:
gamma_nums = [0.00005,0.0001,0.000125,0.00015,0.000175,0.0002,]
C_values= [5000,75000,10000,12500,15000]
x = len(gamma_nums)
y = len(C_values)
count=0

SVR_RMSE_train_list =[]
SVR_RMSE_test_list =[]
SVR_MAPE_train_list =[]
SVR_MAPE_test_list =[]
SVR_par_diff_list=[]
SVR_par_tsize_list=[]
SVR_par_gamma_list=[]
SVR_par_cval_list=[]
SVR_min_vars_tsize_list=[]
SVR_min_vars_gamma_list=[]
SVR_min_vars_cval_list=[]   

print("----progress...----")

for ts in test_size:
    for a in gamma_nums:
        for b in C_values:
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts,random_state=full_count)

            SVR_model = svm.SVR(kernel='rbf',gamma=a,C=b)
            SVR_model.fit(X_train, Y_train)
            
            SVR_prediction_test = SVR_model.predict(X_test)
            SVR_prediction_test = SVR_prediction_test.astype('int')
            
            SVR_train_test = SVR_model.predict(X_train)
            SVR_train_test = SVR_train_test.astype('int')
            
            SVR_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_train_test)).astype('int')
            SVR_MAPE_train = round(np.mean(np.abs((Y_train - SVR_train_test) / Y_train)) * 100 ,2)
            
            SVR_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_prediction_test)).astype('int')
            SVR_MAPE_test = round(np.mean(np.abs((Y_test - SVR_prediction_test) / Y_test)) * 100 ,2)
            
            SVR_RMSE_train_list.append(SVR_RMSE_train)
            SVR_RMSE_test_list.append(SVR_RMSE_test)
            SVR_MAPE_train_list.append(SVR_MAPE_train)
            SVR_MAPE_test_list.append(SVR_MAPE_test)
            
            SVR_RMSE_diff = np.abs((SVR_RMSE_test-SVR_RMSE_train)/SVR_RMSE_test *100)
            
            SVR_par_diff_list.append(SVR_RMSE_diff)
            SVR_par_tsize_list.append(ts)
            SVR_par_gamma_list.append(a)
            SVR_par_cval_list.append(b)     
            
            full_count+=1
            count+=1
            progress = round(count/(x*y*ts_len)*100,2)
            
#             print(" {:.2f}% , case:{} , test_size: {} , Gamma_size: {} ,  C_values: {} , difference(%) : {:.2f}\n "
#             "---  train RMSE : {} , test RMSE : {}\n"
#             "---  train MAPE : {} , test MAPE : {}"
#                   .format(progress,count,ts,a,b,SVR_RMSE_diff,SVR_RMSE_train,SVR_RMSE_test,SVR_MAPE_train,SVR_MAPE_test))
            
for i in range(5):
    val = min(SVR_par_diff_list)
    index = SVR_par_diff_list.index(val)
    SVR_min_vars_tsize_list.append(SVR_par_tsize_list.pop(index))
    SVR_min_vars_gamma_list.append(SVR_par_gamma_list.pop(index))
    SVR_min_vars_cval_list.append(SVR_par_cval_list.pop(index))


In [9]:
print("PARAMETERS FOR BEST SVR...")
for i in range(5):
    print("test_size : {} , Gamma_size: {} ,  C_values: {}".format(SVR_min_vars_tsize_list[i],SVR_min_vars_gamma_list[i],SVR_min_vars_cval_list[i]))

print("\n\n...testing...\n")
progress=0
count=0
SVR_Best_RMSE_train_list=[]
SVR_Best_RMSE_test_list=[]
SVR_Best_MAPE_train_list=[]
SVR_Best_MAPE_test_list=[]
SVR_Best_RMSE_diff_list=[]

SVR_Best_total_data=[]

for i in range(5):
    temp_SVR_Best_RMSE_train_list=[]
    temp_SVR_Best_RMSE_test_list=[]
    temp_SVR_Best_MAPE_train_list=[]
    temp_SVR_Best_MAPE_test_list=[]
    temp_SVR_Best_RMSE_diff_list=[]
    for k in range(test_times):
        ts = SVR_min_vars_tsize_list[i]
        a = SVR_min_vars_gamma_list[i]
        b = SVR_min_vars_cval_list[i]

        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts,random_state=full_count)

        SVR_Best_model = svm.SVR(kernel='rbf',gamma=a,C=b)
        SVR_Best_model.fit(X_train, Y_train)
            
        SVR_Best_prediction_test = SVR_Best_model.predict(X_test)
        SVR_Best_prediction_test = SVR_Best_prediction_test.astype('int')
            
        SVR_Best_train_test = SVR_Best_model.predict(X_train)
        SVR_Best_train_test = SVR_Best_train_test.astype('int')
            
        SVR_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_Best_train_test)).astype('int')
        SVR_Best_MAPE_train = round(np.mean(np.abs((Y_train - SVR_Best_train_test) / Y_train)) * 100 ,2)
            
        SVR_Best_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_Best_prediction_test)).astype('int')
        SVR_Best_MAPE_test = round(np.mean(np.abs((Y_test - SVR_Best_prediction_test) / Y_test)) * 100 ,2)
            
        SVR_Best_RMSE_diff = np.abs((SVR_Best_RMSE_test-SVR_Best_RMSE_train)/SVR_Best_RMSE_test *100)
        
        temp_SVR_Best_RMSE_train_list.append(SVR_Best_RMSE_train)
        temp_SVR_Best_RMSE_test_list.append(SVR_Best_RMSE_test)
        temp_SVR_Best_MAPE_train_list.append(SVR_Best_MAPE_train)
        temp_SVR_Best_MAPE_test_list.append(SVR_Best_MAPE_test)
        temp_SVR_Best_RMSE_diff_list.append(SVR_Best_RMSE_diff)
        full_count+=1  
        count+=1
#         progress = round(count/(test_times*5)*100,2)
#         print(" {:.2f}% , case:{} , test_size: {} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
#                 "---  train RMSE : {} , test RMSE : {}\n"
#                 "---  train MAPE : {} , test MAPE : {}"
#                       .format(progress,count,ts,a,b,SVR_Best_RMSE_diff,SVR_Best_RMSE_train,SVR_Best_RMSE_test,SVR_Best_MAPE_train,SVR_Best_MAPE_test))
    
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_train_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_test_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_train_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_test_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_train_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_test_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_train_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_test_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_diff_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_diff_list))

    
    


PARAMETERS FOR BEST SVR...


NameError: name 'SVR_min_vars_tsize_list' is not defined

In [5]:
SVR1=[]
SVR2=[]
SVR3=[]
SVR4=[]
SVR5=[]

for i in range(50):
    k=i//10
    if k==0:
        SVR1.append(SVR_Best_total_data[i])
    if k==1:
        SVR2.append(SVR_Best_total_data[i])
    if k==2:
        SVR3.append(SVR_Best_total_data[i])
    if k==3:
        SVR4.append(SVR_Best_total_data[i])
    if k==4:
        SVR5.append(SVR_Best_total_data[i])

    
SVR_s1 = pd.Series(SVR1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

SVR_s2 = pd.Series(SVR2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
SVR_s3 = pd.Series(SVR3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
SVR_s4 = pd.Series(SVR4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
SVR_s5 = pd.Series(SVR5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')

SVR_best_total_dataframe_rbf = pd.concat([SVR_s1,SVR_s2,SVR_s3,SVR_s4,SVR_s5],axis=1)
print("SVR best 5 models")
SVR_best_total_dataframe_rbf

NameError: name 'SVR_Best_total_data' is not defined

### EXTRA... SVM LINEAR KERNEL

In [12]:
# gamma_nums = [0.001,0.01,0.1,1]
# C_values= [10,100,1000,10000]
# x = len(gamma_nums)
# y = len(C_values)
# count=0
# # Random Forest Relaxed Search
# count=0

# # best_SVR_RMSE_diff=100
# # best_SVR_RMSE_train=0
# # best_SVR_RMSE_test=0
# # best_SVR_MAPE_train=0
# # best_SVR_MAPE_test=0
# SVR_RMSE_train_list =[]
# SVR_RMSE_test_list =[]
# SVR_MAPE_train_list =[]
# SVR_MAPE_test_list =[]
# SVR_par_diff_list=[]
# SVR_par_tsize_list=[]
# SVR_par_gamma_list=[]
# SVR_par_cval_list=[]
# SVR_min_vars_tsize_list=[]
# SVR_min_vars_gamma_list=[]
# SVR_min_vars_cval_list=[]   

# print("----progress...----")

# for ts in test_size:
#     for a in gamma_nums:
#         for b in C_values:

#             X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

#             SVR_model = svm.SVR(kernel='linear',gamma=a,C=b)
#             SVR_model.fit(X_train, Y_train)
            
#             SVR_prediction_test = SVR_model.predict(X_test)
#             SVR_prediction_test = SVR_prediction_test.astype('int')
            
#             SVR_train_test = SVR_model.predict(X_train)
#             SVR_train_test = SVR_train_test.astype('int')
            
#             SVR_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_train_test)).astype('int')
#             SVR_MAPE_train = round(np.mean(np.abs((Y_train - SVR_train_test) / Y_train)) * 100 ,2)
            
#             SVR_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_prediction_test)).astype('int')
#             SVR_MAPE_test = round(np.mean(np.abs((Y_test - SVR_prediction_test) / Y_test)) * 100 ,2)
            
#             SVR_RMSE_train_list.append(SVR_RMSE_train)
#             SVR_RMSE_test_list.append(SVR_RMSE_test)
#             SVR_MAPE_train_list.append(SVR_MAPE_train)
#             SVR_MAPE_test_list.append(SVR_MAPE_test)
            
#             SVR_RMSE_diff = np.abs((SVR_RMSE_test-SVR_RMSE_train)/SVR_RMSE_test *100)
            
#             SVR_par_diff_list.append(SVR_RMSE_diff)
#             SVR_par_tsize_list.append(ts)
#             SVR_par_gamma_list.append(a)
#             SVR_par_cval_list.append(b)
         
                
#             count+=1
#             progress = round(count/(x*y*ts_len)*100,2)
            
#             print(" {:.2f}% , case:{} , test_size: {} , Gamma_size: {} ,  C_values: {} , difference(%) : {:.2f}\n "
#             "---  train RMSE : {} , test RMSE : {}\n"
#             "---  train MAPE : {} , test MAPE : {}"
#                   .format(progress,count,ts,a,b,SVR_RMSE_diff,SVR_RMSE_train,SVR_RMSE_test,SVR_MAPE_train,SVR_MAPE_test))
            


# for i in range(5):
#     val = min(SVR_par_diff_list)
#     index = SVR_par_diff_list.index(val)
#     SVR_min_vars_tsize_list.append(SVR_par_tsize_list.pop(index))
#     SVR_min_vars_gamma_list.append(SVR_par_gamma_list.pop(index))
#     SVR_min_vars_cval_list.append(SVR_par_cval_list.pop(index))


In [13]:
# print("PARAMETERS FOR BEST SVR...")
# for i in range(5):
#     print("test_size : {} , Gamma_size: {} ,  C_values: {}".format(SVR_min_vars_tsize_list[i],SVR_min_vars_gamma_list[i],SVR_min_vars_cval_list[i]))

# print("\n\n...testing...\n")
# test_times=5
# progress=0
# count=0
# SVR_Best_RMSE_train_list=[]
# SVR_Best_RMSE_test_list=[]
# SVR_Best_MAPE_train_list=[]
# SVR_Best_MAPE_test_list=[]
# SVR_Best_RMSE_diff_list=[]

# SVR_Best_total_data=[]

# for i in range(5):
#     temp_SVR_Best_RMSE_train_list=[]
#     temp_SVR_Best_RMSE_test_list=[]
#     temp_SVR_Best_MAPE_train_list=[]
#     temp_SVR_Best_MAPE_test_list=[]
#     temp_SVR_Best_RMSE_diff_list=[]
#     for k in range(test_times):
#         ts = SVR_min_vars_tsize_list[i]
#         a = SVR_min_vars_gamma_list[i]
#         b = SVR_min_vars_cval_list[i]

#         X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

#         SVR_Best_model = svm.SVR(kernel='linear',gamma=a,C=b)
#         SVR_Best_model.fit(X_train, Y_train)
            
#         SVR_Best_prediction_test = SVR_Best_model.predict(X_test)
#         SVR_Best_prediction_test = SVR_Best_prediction_test.astype('int')
            
#         SVR_Best_train_test = SVR_Best_model.predict(X_train)
#         SVR_Best_train_test = SVR_Best_train_test.astype('int')
            
#         SVR_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_Best_train_test)).astype('int')
#         SVR_Best_MAPE_train = round(np.mean(np.abs((Y_train - SVR_Best_train_test) / Y_train)) * 100 ,2)
            
#         SVR_Best_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_Best_prediction_test)).astype('int')
#         SVR_Best_MAPE_test = round(np.mean(np.abs((Y_test - SVR_Best_prediction_test) / Y_test)) * 100 ,2)
            
#         SVR_Best_RMSE_diff = np.abs((SVR_Best_RMSE_test-SVR_Best_RMSE_train)/SVR_Best_RMSE_test *100)
        
#         temp_SVR_Best_RMSE_train_list.append(SVR_Best_RMSE_train)
#         temp_SVR_Best_RMSE_test_list.append(SVR_Best_RMSE_test)
#         temp_SVR_Best_MAPE_train_list.append(SVR_Best_MAPE_train)
#         temp_SVR_Best_MAPE_test_list.append(SVR_Best_MAPE_test)
#         temp_SVR_Best_RMSE_diff_list.append(SVR_Best_RMSE_diff)
            

#         count+=1
#         progress = round(count/(test_times*5)*100,2)
#         print(" {:.2f}% , case:{} , test_size: {} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
#                 "---  train RMSE : {} , test RMSE : {}\n"
#                 "---  train MAPE : {} , test MAPE : {}"
#                       .format(progress,count,ts,a,b,SVR_Best_RMSE_diff,SVR_Best_RMSE_train,SVR_Best_RMSE_test,SVR_Best_MAPE_train,SVR_Best_MAPE_test))
    
#     SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_train_list))
#     SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_test_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_train_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_test_list))
#     SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_train_list))
#     SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_test_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_train_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_test_list))
#     SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_diff_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_diff_list))


In [14]:
# SVR1=[]
# SVR2=[]
# SVR3=[]
# SVR4=[]
# SVR5=[]

# for i in range(50):
#     k=i//10
#     if k==0:
#         SVR1.append(SVR_Best_total_data[i])
#     if k==1:
#         SVR2.append(SVR_Best_total_data[i])
#     if k==2:
#         SVR3.append(SVR_Best_total_data[i])
#     if k==3:
#         SVR4.append(SVR_Best_total_data[i])
#     if k==4:
#         SVR5.append(SVR_Best_total_data[i])

    
# SVR_s1 = pd.Series(SVR1,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 1')

# SVR_s2 = pd.Series(SVR2,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 2')
# SVR_s3 = pd.Series(SVR3,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 3')
# SVR_s4 = pd.Series(SVR4,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 4')
# SVR_s5 = pd.Series(SVR5,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 5')

# SVR_best_total_dataframe_linear = pd.concat([SVR_s1,SVR_s2,SVR_s3,SVR_s4,SVR_s5],axis=1)
# print("SVR best 5 models")
# SVR_best_total_dataframe_linear

### 3. Linear Regression ... <br>
what parameters are there here?

In [15]:
times=10
count=0

LR_RMSE_train_list =[]
LR_RMSE_test_list =[]
LR_MAPE_train_list =[]
LR_MAPE_test_list =[]
LR_par_diff_list=[]
LR_par_tsize_list=[]
LR_min_vars_tsize_list=[] 

print("----progress...----")

for ts in test_size:
    for i in range(times):
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts,random_state=full_count)

        LR_model=LinearRegression()
        LR_model.fit(X_train, Y_train)
        LR_prediction_test = LR_model.predict(X_test)
        LR_prediction_test = LR_prediction_test.astype('int')
        LR_train_test = LR_model.predict(X_train)
        LR_train_test = LR_train_test.astype('int')
    
        LR_RMSE_train =np.sqrt(mean_squared_error(Y_train, LR_train_test)).astype('int')
        LR_MAPE_train = round(np.mean(np.abs((Y_train - LR_train_test) / Y_train)) * 100 ,2)
            
        LR_RMSE_test =np.sqrt(mean_squared_error(Y_test, LR_prediction_test)).astype('int')
        LR_MAPE_test = round(np.mean(np.abs((Y_test - LR_prediction_test) / Y_test)) * 100 ,2)
        
        LR_RMSE_train_list.append(LR_RMSE_train)
        LR_RMSE_test_list.append(LR_RMSE_test)
        LR_MAPE_train_list.append(LR_MAPE_train)
        LR_MAPE_test_list.append(LR_MAPE_test)
            
        LR_RMSE_diff = np.abs((LR_RMSE_test-LR_RMSE_train)/LR_RMSE_test *100)
            
        LR_par_diff_list.append(LR_RMSE_diff)
        LR_par_tsize_list.append(ts)

         
        full_count+=1     
        count+=1
        progress = round(count/(ts_len*times)*100,2)
            
#         print(" {:.2f}% , case:{} , test_size: {} , difference(%) : {:.2f}\n "
#         "---  train RMSE : {} , test RMSE : {}\n"
#         "---  train MAPE : {} , test MAPE : {}"
#                 .format(progress,count,ts,LR_RMSE_diff,LR_RMSE_train,LR_RMSE_test,LR_MAPE_train,LR_MAPE_test))



for i in range(5):
    val = min(LR_par_diff_list)
    index = LR_par_diff_list.index(val)
    LR_min_vars_tsize_list.append(LR_par_tsize_list.pop(index))


----progress...----


In [16]:
print("PARAMETERS FOR BEST LR...")
for i in range(5):
    print("test_size : {} ".format(LR_min_vars_tsize_list[i]))

print("...testing...")

progress=0
count=0
LR_Best_RMSE_train_list=[]
LR_Best_RMSE_test_list=[]
LR_Best_MAPE_train_list=[]
LR_Best_MAPE_test_list=[]
LR_Best_RMSE_diff_list=[]

LR_Best_total_data=[]

for i in range(5):
    temp_LR_Best_RMSE_train_list=[]
    temp_LR_Best_RMSE_test_list=[]
    temp_LR_Best_MAPE_train_list=[]
    temp_LR_Best_MAPE_test_list=[]
    temp_LR_Best_RMSE_diff_list=[]
    
    for k in range(test_times):
        ts = LR_min_vars_tsize_list[i]
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts,random_state=full_count)

        LR_Best_model=LinearRegression()
        LR_Best_model.fit(X_train, Y_train)
        LR_Best_prediction_test = LR_Best_model.predict(X_test)
        LR_Best_prediction_test = LR_Best_prediction_test.astype('int')
        LR_Best_train_test = LR_Best_model.predict(X_train)
        LR_Best_train_test = LR_Best_train_test.astype('int')

        LR_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, LR_Best_train_test)).astype('int')
        LR_Best_MAPE_train = round(np.mean(np.abs((Y_train - LR_Best_train_test) / Y_train)) * 100 ,2)

        LR_Best_RMSE_test = np.sqrt(mean_squared_error(Y_test, LR_Best_prediction_test)).astype('int')
        LR_Best_MAPE_test = round(np.mean(np.abs((Y_test - LR_Best_prediction_test) / Y_test)) * 100 ,2)

        LR_Best_RMSE_diff = np.abs((LR_Best_RMSE_test-LR_Best_RMSE_train)/LR_Best_RMSE_test *100)
        
        temp_LR_Best_RMSE_train_list.append(LR_Best_RMSE_train)
        temp_LR_Best_RMSE_test_list.append(LR_Best_RMSE_test)
        temp_LR_Best_MAPE_train_list.append(LR_Best_MAPE_train)
        temp_LR_Best_MAPE_test_list.append(LR_Best_MAPE_test)
        temp_LR_Best_RMSE_diff_list.append(LR_Best_RMSE_diff)
        
        full_count+=1
        count+=1
        
#         progress = round(count/(test_times*5)*100,2)
#         print(" {:.2f}% , case:{} , test_size: {} , difference(%) : {:.2f}\n "
#                 "---  train RMSE : {} , test RMSE : {}\n"
#                 "---  train MAPE : {} , test MAPE : {}"
#                       .format(progress,count,ts,LR_Best_RMSE_diff,LR_Best_RMSE_train,LR_Best_RMSE_test,LR_Best_MAPE_train,LR_Best_MAPE_test))
    
    LR_Best_total_data.append(mean(temp_LR_Best_RMSE_train_list))
    LR_Best_total_data.append(mean(temp_LR_Best_RMSE_test_list))
    LR_Best_total_data.append(std(temp_LR_Best_RMSE_train_list))
    LR_Best_total_data.append(std(temp_LR_Best_RMSE_test_list))
    LR_Best_total_data.append(mean(temp_LR_Best_MAPE_train_list))
    LR_Best_total_data.append(mean(temp_LR_Best_MAPE_test_list))
    LR_Best_total_data.append(std(temp_LR_Best_MAPE_train_list))
    LR_Best_total_data.append(std(temp_LR_Best_MAPE_test_list))
    LR_Best_total_data.append(mean(temp_LR_Best_RMSE_diff_list))
    LR_Best_total_data.append(std(temp_LR_Best_RMSE_diff_list))
    


PARAMETERS FOR BEST LR...
test_size : 0.3 
test_size : 0.3 
test_size : 0.3 
test_size : 0.3 
test_size : 0.3 
...testing...


In [17]:


LR1=[]
LR2=[]
LR3=[]
LR4=[]
LR5=[]

for i in range(50):
    k=i//10
    if k==0:
        LR1.append(LR_Best_total_data[i])
    if k==1:
        LR2.append(LR_Best_total_data[i])
    if k==2:
        LR3.append(LR_Best_total_data[i])
    if k==3:
        LR4.append(LR_Best_total_data[i])
    if k==4:
        LR5.append(LR_Best_total_data[i])
    
LR_s1 = pd.Series(LR1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

LR_s2 = pd.Series(LR2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
LR_s3 = pd.Series(LR3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
LR_s4 = pd.Series(LR4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
LR_s5 = pd.Series(LR5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')

LR_best_total_dataframe = pd.concat([LR_s1,LR_s2,LR_s3,LR_s4,LR_s5],axis=1)

LR_best_total_dataframe

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,1597.200000,1578.300000,1582.800000,1614.100000,1576.400000
RMSE test Mean,1675.500000,1714.700000,1752.800000,1633.900000,1776.200000
RMSE train Stdev,51.795367,40.383289,57.565267,46.237323,54.666626
RMSE test Stdev,125.070580,89.977831,190.310693,108.814935,230.536244
MAPE train Mean,9.046000,9.022000,9.063000,9.103000,8.951000
MAPE test Mean,9.246000,9.226000,9.293000,9.262000,9.648000
MAPE train Stdev,0.203774,0.132424,0.247994,0.183306,0.258977
MAPE test Stdev,0.341004,0.287305,0.708181,0.370912,0.757440
RMSE diff(%) Mean,9.489835,7.748567,11.747594,8.497135,11.193396
RMSE diff(%) Stdev,5.333568,6.833080,8.867700,4.981137,10.942367


# Final Values

In [31]:
RF_best_total_dataframe

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,1803.900000,1274.400000,912.100000,2741.700000,1828.900000
RMSE test Mean,1810.400000,1749.200000,1222.500000,2765.300000,2038.300000
RMSE train Stdev,11.193302,37.739104,16.973214,74.762357,44.706711
RMSE test Stdev,92.152265,343.614260,251.972717,291.598371,269.532948
MAPE train Mean,11.383000,7.852000,5.411000,17.562000,11.643000
MAPE test Mean,11.539000,8.733000,6.625000,17.006000,12.192000
MAPE train Stdev,0.100404,0.152892,0.040112,0.445439,0.260617
MAPE test Stdev,0.229846,0.375847,0.230835,0.855783,0.824388
RMSE diff(%) Mean,4.692400,24.018964,22.947576,11.394745,9.458200
RMSE diff(%) Stdev,3.266955,16.177747,11.991082,5.981762,9.974508


In [19]:
SVR_best_total_dataframe_rbf

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,1418.100000,1422.800000,1344.300000,1485.100000,912.100000
RMSE test Mean,1566.300000,1503.600000,1575.700000,1568.700000,1441.500000
RMSE train Stdev,88.802534,103.435777,83.682794,76.616513,23.209696
RMSE test Stdev,225.923460,291.948352,211.652569,178.052268,316.011788
MAPE train Mean,6.163000,6.017000,5.869000,6.163000,4.478000
MAPE test Mean,6.952000,6.626000,6.648000,7.040000,6.084000
MAPE train Stdev,0.136605,0.090559,0.105873,0.113846,0.092607
MAPE test Stdev,0.363257,0.419051,0.477175,0.273861,0.282779
RMSE diff(%) Mean,16.760599,18.647974,15.811527,10.993868,33.355786
RMSE diff(%) Stdev,10.687624,13.226650,12.603149,9.632906,15.604659


In [20]:
LR_best_total_dataframe

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,1597.200000,1578.300000,1582.800000,1614.100000,1576.400000
RMSE test Mean,1675.500000,1714.700000,1752.800000,1633.900000,1776.200000
RMSE train Stdev,51.795367,40.383289,57.565267,46.237323,54.666626
RMSE test Stdev,125.070580,89.977831,190.310693,108.814935,230.536244
MAPE train Mean,9.046000,9.022000,9.063000,9.103000,8.951000
MAPE test Mean,9.246000,9.226000,9.293000,9.262000,9.648000
MAPE train Stdev,0.203774,0.132424,0.247994,0.183306,0.258977
MAPE test Stdev,0.341004,0.287305,0.708181,0.370912,0.757440
RMSE diff(%) Mean,9.489835,7.748567,11.747594,8.497135,11.193396
RMSE diff(%) Stdev,5.333568,6.833080,8.867700,4.981137,10.942367
